![image](images/um_logo.png)

# Computación II


# ***Generadores***
https://wiki.python.org/moin/Generators

Los generadores son un concepto importante en la programación que se utiliza para crear secuencias de valores de manera eficiente y con bajo consumo de memoria. En términos generales, un generador es una función especial que permite generar valores sobre la marcha, en lugar de calcular y almacenar todos los valores en la memoria de antemano. Esto es especialmente útil cuando se trabaja con secuencias potencialmente grandes o infinitas de datos.

En lugar de devolver todos los valores a la vez en una lista o una estructura de datos similar, un generador produce un valor cada vez que se solicita y luego pausa su ejecución hasta que se solicita el siguiente valor. Esto reduce significativamente el uso de memoria y permite trabajar con secuencias potencialmente ilimitadas de datos.

En __Python__, los generadores son implementados a través de funciones generadoras. Una función generadora se define utilizando la palabra clave `yield` en lugar de `return`. Cuando una función generadora se llama, no se ejecuta completamente de una sola vez. En su lugar, produce un objeto generador que puede ser utilizado para obtener valores uno a uno.



In [ ]:
import time

def generador_simple():
    print('Vamos a ejecutar el yield 1')
    yield 1
    #hacer lo que quiera
    print('Vamos a ejecutar el yield 2')
    yield 2
    print('Vamos a ejecutar el yield 3 con mucho tiempo de ejecución')
    time.sleep(3)
    yield 3

gen = generador_simple()

for valor in gen:
    print(valor)


El generador `generador_simple()` produce valores del 1 al 3 utilizando yield. Cada vez que se llama a yield, el generador pausa su ejecución y devuelve el valor, y luego se reanuda desde donde se quedó en la siguiente llamada.

Los generadores en Python son especialmente útiles cuando se trabaja con grandes conjuntos de datos o cuando se necesita generar secuencias potencialmente infinitas, como números primos o secuencias de Fibonacci. También son una forma eficiente de procesar grandes archivos de datos línea por línea sin cargar todo el archivo en memoria.


In [ ]:
def calcular_pi():# aproximación con serie de Leibniz 
    suma = 0
    signo = 1
    denominador = 1

    while True:
        termino = signo * (1 / denominador)
        suma += termino
        yield 4 * suma  # Multiplicamos por 4 para obtener una mejor aproximación de π
        signo *= -1
        denominador += 2

# Crear un generador para calcular π
generador_pi = calcular_pi()

# Imprimir los primeros 10 decimales de π
for _ in range(100000):
    next(generador_pi)
    
print(next(generador_pi))


In [ ]:
import math

def calcular_e():
    n = 0
    e = 0
    #while n < 10:
    while True:
        e += 1/math.factorial(n)
        n += 1
        yield e #pausa.
        
    #yield 'Terminé'
        
generador_e = calcular_e()

for _ in range(20):
    print(next(generador_e))
    


### Delegación de generadores
La palabra clave from seguida de yield se usa para delegar el control a otro generador en Python. Esto permite que un generador "padre" delegue parte de su trabajo a otro generador "hijo" y obtenga los valores generados por el generador hijo. Es una característica avanzada de generadores que se utiliza en situaciones donde se desea componer varios generadores en uno solo

In [ ]:
def generador_hijo():
    yield 'A'
    yield 'B'

def generador_padre():
    yield 'X'
    yield from generador_hijo()
    yield 'Y'

# Usar el generador padre
for valor in generador_padre():
    print(valor)
    
    
#['X', ['A', 'B'], 'Y'] analogía con una lista


En este ejemplo, tenemos dos generadores: `generador_hijo` y `generador_padre`. El `generador_padre` utiliza `yield from generador_hijo()` para delegar parte de su trabajo al `generador_hijo`. Cuando el generador_padre se ejecuta, produce los valores __'X'__, luego __'A'__ y __'B'__ (que provienen del generador_hijo) y por último, __'Y'__. La palabra clave `yield from` se utiliza para _"desenrollar"_ el `generador_hijo` dentro del `generador_padre`.

### Un mecanismo rústico de asincronismo

In [ ]:
import time
import random

# Función generadora para simular la lectura de sensores
def leer_sensores():
    while True:
        sensor1 = random.randint(0, 100)
        print('Sensor 1')
        time.sleep(1)
        yield sensor1 #Si quisiera que la lectura de sensor1 y sensor2 fuese atómica, deberia comentar esta línea
        
        sensor2 = random.randint(0, 100)
        print('Sensor 2')
        time.sleep(1)
        yield sensor2
        
        sensor3 = random.randint(0, 100)
        print('Sensor 3')
        time.sleep(1)
        yield sensor3
        
        sensor4 = random.randint(0, 100)
        print('Sensor 4')
        time.sleep(1)
        yield sensor4
        

# Función generadora para simular el envío de datos
def enviar_datos():
    while True:
        datos = yield
        print("Enviando datos:", datos)
        time.sleep(1)

# Crear instancias de los generadores
generador_lectura = leer_sensores()
generador_envio = enviar_datos()
next(generador_envio)  # Iniciar el generador de envío

# Alternar entre los generadores en un bucle infinito
while True:
    lectura = next(generador_lectura)
    generador_envio.send(lectura1 )


# Funcionamiento interno de los generadores en Python

## 1. Estructura de memoria

Cuando se crea un generador, Python asigna una estructura de memoria especial que incluye:

- **Frame de ejecución**: Similar a una función normal, pero con estado persistente.
- **Código del generador**: Las instrucciones bytecode de la función generadora.
- **Estado del generador**: Indica si está en ejecución, suspendido, o terminado.
- **Valor de yield**: Almacena el último valor producido por `yield`.

## 2. Proceso de ejecución

1. **Creación**: 
   - Al llamar a la función generadora, se crea el objeto generador pero no se ejecuta el código.
   - Se inicializa el frame de ejecución con las variables locales.

2. **Primera llamada a `next()` o `send(None)`**:
   - Se inicia la ejecución del código del generador.
   - El generador se ejecuta hasta encontrar un `yield`.
   - El estado se guarda (incluyendo la posición en el código) y se suspende la ejecución.
   - El valor yielded se devuelve al llamador.

3. **Llamadas subsiguientes**:
   - Se reanuda la ejecución desde el punto donde se suspendió.
   - Se continúa hasta el siguiente `yield` o hasta que termine.

## 3. Manejo del stack

- Los generadores utilizan un mecanismo llamado "stack unwinding" (desenrollado de pila).
- Cuando se llega a un `yield`, el frame de ejecución actual se guarda y se retira del stack.
- Cuando se reanuda, el frame se restaura en el stack para continuar la ejecución.

## 4. Implementación de `send()`

- `send()` permite inyectar un valor en el generador.
- Internamente, coloca el valor enviado en una ubicación especial del frame del generador.
- Cuando se reanuda la ejecución, este valor se usa como el resultado de la expresión `yield`.

## 5. Optimizaciones

- Python utiliza una técnica llamada "trampolín" para generadores encadenados.
- Esto permite que los generadores anidados sean tan eficientes como un solo generador.

## 6. Generadores asíncronos

- Usan `async def` y `await`.
- Funcionan de manera similar, pero están diseñados para trabajar con bucles de eventos asíncronos.

# **asyncio**
El módulo `asyncio` en Python es una biblioteca que proporciona soporte para la programación asíncrona y concurrente, utilizando la sintaxis de `async` y `await`. Esta biblioteca es parte de la biblioteca estándar de Python y se utiliza para escribir código que puede realizar tareas de manera concurrente sin bloquear el hilo principal de ejecución. `asyncio` es especialmente útil para aplicaciones que requieren operaciones de entrada/salida (E/S) no bloqueantes, como servidores web, clientes de red, tareas de E/S intensivas y más.

## Algunos conceptos clave de __asyncio__
1. __Corrutinas Asíncronas:__ `asyncio` se basa en el uso de funciones asincrónicas o "corrutinas asíncronas", que se definen utilizando la palabra clave `async` antes de la definición de una función. Las corrutinas asíncronas se pueden suspender y reanudar, lo que permite que otras tareas se ejecuten mientras una espera E/S u otras operaciones asincrónicas.

2. __await:__ Dentro de una función asincrónica, es posible utilizar la palabra clave `await` para esperar la finalización de una tarea asincrónica, como una operación de E/S o una tarea larga. El uso de `await` permite que el control se devuelva al bucle de eventos de `asyncio` mientras la tarea esperada se completa.

3. __Bucle de Eventos:__ `asyncio` utiliza un bucle de eventos para gestionar múltiples tareas asincrónicas de manera concurrente. Este bucle de eventos se encarga de planificar y ejecutar tareas, así como de manejar la comunicación entre ellas.

4. __Tareas:__ En `asyncio`, las tareas son unidades de trabajo asincrónico. Es posible crear y ejecutar múltiples tareas concurrentemente, lo que facilita la escritura de código concurrente y paralelo.

5. __Gestión de E/S No Bloqueante:__ `asyncio` es particularmente eficaz en la gestión de operaciones de E/S no bloqueantes, como la comunicación en red o la lectura/escritura de archivos. Esto permite que las aplicaciones respondan de manera eficiente a muchas solicitudes simultáneas sin bloquear el hilo principal.

6. __Protocolos y Controladores de Eventos:__ `asyncio` proporciona una serie de protocolos y controladores de eventos que permiten trabajar con diferentes tipos de conexiones de red, como TCP, UDP y más.

In [ ]:
import asyncio

# Definir una función asincrónica
async def hola_mundo():
    print("Esperando...")
    await asyncio.sleep(1)  # Espera durante 1 segundo (simulando una operación lenta)
    print("¡Hola, mundo!")

# Crear un bucle de eventos de asyncio
async def main():
    await hola_mundo()  # Ejecutar la función asincrónica

# Ejecutar el bucle de eventos de asyncio
asyncio.run(main())


In [ ]:
import asyncio
import time

async def say_after(delay, what):
    await asyncio.sleep(delay)
    print(what)

async def main():
    print(f"started at {time.strftime('%X')}")

    await say_after(1, 'hello')
    await say_after(2, 'world')

    print(f"finished at {time.strftime('%X')}")

asyncio.run(main())

### Creando tareas

In [ ]:
import asyncio
import time


async def say_after(delay, what):
    await asyncio.sleep(delay)
    print(what)

async def main():
    task1 = asyncio.create_task(
        say_after(1, 'hello'))

    task2 = asyncio.create_task(
        say_after(2, 'world'))

    print(f"started at {time.strftime('%X')}")

    # Wait until both tasks are completed (should take
    # around 2 seconds.)
    await task1
    await task2

    print(f"finished at {time.strftime('%X')}")
    
asyncio.run(main())

In [ ]:
import asyncio

# Definir una función asincrónica
async def tarea_espera():
    print("Inicio de la tarea de espera")
    await asyncio.sleep(2)  # Espera durante 2 segundos (simulando una operación lenta)
    print("Fin de la tarea de espera")

# Definir una función asincrónica
async def tarea_impresion():
    print("Inicio de la tarea de impresión")
    await asyncio.sleep(5)
    print("Hola, soy una tarea de impresión")
    print("Fin de la tarea de impresión")

# Definir una función asincrónica
async def tarea_suma():
    print("Inicio de la tarea de suma")
    await asyncio.sleep(3)
    resultado = 10 + 20
    print(f"El resultado de la suma es {resultado}")
    print("Fin de la tarea de suma")

# Crear un bucle de eventos de asyncio
async def main():
    tarea1 = asyncio.create_task(tarea_espera())  # Crear una tarea asincrónica
    tarea2 = asyncio.create_task(tarea_impresion())
    tarea3 = asyncio.create_task(tarea_suma())

    await tarea1  # Esperar a que la tarea de espera termine
    await tarea2  # Esperar a que la tarea de impresión termine
    await tarea3  # Esperar a que la tarea de suma termine

# Ejecutar el bucle de eventos de asyncio
asyncio.run(main())

### Un ejemplo con _aiohttp_

In [ ]:
import asyncio
import aiohttp

async def obtener_url(url):
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as respuesta:
            contenido = await respuesta.text()
            print(f"URL: {url}, Longitud del contenido: {len(contenido)}")

async def main():
    urls = [
        "https://www.ejemplo.com",
        "https://www.python.org",
        "https://www.google.com"
    ]

    # Crear tareas asincrónicas para obtener las URL de manera concurrente
    tareas = [obtener_url(url) for url in urls]

    # Esperar a que se completen todas las tareas
    await asyncio.gather(*tareas) #(tareas[0], tareas[1], tareas[3], ....)

# Ejecutar el bucle de eventos de asyncio
asyncio.run(main())


1. Importamos las bibliotecas `asyncio` y `aiohttp`. `aiohttp` es una biblioteca asincrónica que se utiliza para realizar solicitudes web de manera eficiente en un contexto asincrónico.

2. Definimos una función asincrónica `obtener_url(url)` que utiliza `aiohttp` para realizar una solicitud `GET` a una URL dada y luego muestra la longitud del contenido de la respuesta.

3. En la función `main()`, creamos una lista de URLs que deseamos solicitar de manera concurrente.

4. Luego, creamos una lista de tareas asincrónicas, una por cada URL en la lista, que llamará a `obtener_url(url)`.

5. Utilizamos `asyncio.gather(*tareas)` para ejecutar todas las tareas de manera concurrente y esperar a que se completen.


| Característica                 | asyncio                            | threading                        | multiprocessing                  |
|---------------------------------|-----------------------------------|----------------------------------|-----------------------------------|
| **Modelo de Concurrencia**     | Programación Asincrónica (async/await) | Programación Concurrente (threads) | Programación Concurrente (procesos) |
| **Escalamiento**               | Adecuado para alto número de conexiones asincrónicas | Limitado por GIL (Global Interpreter Lock) | Adecuado para CPU-bound tasks     |
| **Tipo de Tareas**             | Tareas asincrónicas                | Hilos                            | Procesos                          |
| **Uso Eficiente de Recursos**  | Bajo consumo de memoria            | Mayor consumo de memoria          | Mayor consumo de memoria y CPU    |
| **Facilidad de Uso**           | Uso de async/await, menos propenso a problemas de concurrencia | Sincronización manual, susceptible a problemas de concurrencia | Sincronización manual, más seguro contra problemas de concurrencia |
| **Depuración**                 | Requiere herramientas de depuración asincrónica | Herramientas de depuración estándar | Herramientas de depuración estándar |
| **Aplicaciones Típicas**       | Aplicaciones con E/S no bloqueantes, servidores web asincrónicos | Aplicaciones con hilos de interfaz de usuario (UI), aplicaciones con múltiples hilos | Aplicaciones que requieren cálculos intensivos en CPU, procesamiento paralelo |


Un tutorial
https://medium.com/@ar.aldhafeeri11/part-i-python-asyncio-deep-dive-b639f8d4bc60